# Data analysis of trip data from “Capital Bikeshare”

**Preface**

"Capital Bikeshare" is a bikesharing service in Washington D.C. and surrounding communities with currently more than 500 stations and over 4,000 available bicycles. More information about how the program works and an overview of the stations can be found at
<a href="https://www.capitalbikeshare.com">www.capitalbikeshare.com</a>.

The dataset originally came from the Washington D.C. Open Data Portal:
<a href="https://opendata.dc.gov/datasets/capital-bike-share-locations">Capital Bikeshare Locations</a>.
This study was first created during my bachelor studies in 2019. I have now reworked and improved it, since many functions from back then are no longer useable. Unfortunately the original dataset is no longer available. 

The analysis is based on the years 2015 to 2017 using the programming language Python. For visualization this Jupyter Notebook is used. To process the data additional Python modules were installed.

The cost overview for Members and Casual users, which is added as images, reflects the current prices in 2025 and no longer those from 2019 when I originally worked on the project. It is also possible that prices have since decreased or increased.

In [ ]:
# core libs
!pip install --upgrade nbformat pandas holidays plotly

# widget stack (classic notebook compatible)
!pip install "notebook==6.1.5" "widgetsnbextension==3.*" "ipywidgets==7.*" anywidget

# enable widgets in classic Notebook
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

## Import, Clean and Enhance Dataset

In [ ]:
import pandas as pd

# path to dataset
data_path = 'data/'

# list of zipped data files
datas = ['2015-capitalbikeshare-tripdata.zip', '2016-capitalbikeshare-tripdata.zip', '2017-capitalbikeshare-tripdata.zip']

# columns to parse as dates
parse_to_dates = ['Start date', 'End date']

df_list = []

# read and append each file to a list
for data in datas:
    df = pd.read_csv(data_path + data, parse_dates=parse_to_dates, compression='zip')
    print(f"Added: \"{data}\" [{len(df)}]")
    df_list.append(df)

# correct incorrect indexes
df_trips = pd.concat(df_list, ignore_index=True)
print("Total records:", len(df_trips))

In [ ]:
# detect NANs
df_corrupted = df_trips[df_trips.isna().any(axis=1)]

# remove NANs
df_trips.dropna(inplace=True)

if len(df_corrupted.index) > 0:
    print("Removed corrupted records:")
    display
else:
    print("No corrupted records found.")

In [ ]:
df_trips.head()

In [ ]:
import holidays

# Add Duration column in seconds
df_trips['Duration'] = (df_trips['End date'] - df_trips['Start date']).dt.total_seconds() 

# Add Weekday column (Mon, Tue, Wed, Thu, Fri, Sat, Sun)
df_trips['Weekday'] = df_trips['Start date'].dt.strftime('%a')

# Add Holiday column (1 if holiday, 0 otherwise)
us_holidays = holidays.US()
df_trips['Holiday'] = df_trips['Start date'].apply(lambda date:date in us_holidays and '1' or '0')
df_trips

## Filter Dataset

The focus is on analyzing bike usage so records that could distort the statistics are removed. This includes very short trips, such as when a bike was only taken out and immediately docked again. Also very long trips, for example when a bike was rented for an entire night. Also rarely used and new stations are filtered out, since they play little role in the analysis.

In [ ]:
# Remove trips shorter than 60 seconds
countBefore = len(df_trips.index)
df_trips = df_trips[~(df_trips['Duration'] < 60)] 
countAfter = len(df_trips.index)
print("Remove trips shorter than 60 seconds: %d" % (countBefore-countAfter))

# filter trips with the same start and end station and duration less than 120 seconds
countBefore = countAfter
df_trips = df_trips[~((df_trips['Duration'] < 120) & (df_trips['Start station number'] == df_trips['End station number']))]
countAfter = len(df_trips.index)
print("Remove trips shorter than 120 seconds with same start and end station: %d" % (countBefore-countAfter))


# filter trips with duration longer than 20 hours 
countBefore = countAfter
df_trips = df_trips[~(df_trips['Duration'] > 20*60*60)]
countAfter = len(df_trips.index)
print("Remove trips longer than 20 hours: %d" % (countBefore-countAfter))

# filter stations with less than 5000 starts
countBefore = countAfter

# get counts of starts per station
station_counts = df_trips.groupby('Start station number').size()

#filter stations with less than 5000 starts
df_trips = df_trips[~df_trips['Start station number'].isin(station_counts[station_counts < 5000].keys())]
countAfter = len(df_trips.index)
print("Remove stations with less than 5000 starts: %d" % (countBefore-countAfter))

After applying the filters the dataset is prepared for further analysis.

## First Impression of the Dataset

In [ ]:
# check number of trips per year
df_trips.groupby(df_trips['Start date'].dt.year).size()

Use of bike-sharing services increased between 2015 and 2017.

In [ ]:
# check top 10 start stations
df_trips.groupby('Start station').size().sort_values(ascending=False).head(10)

<img src="img/washington_dc_map.png"/>
Most of the busiest stations are located in the center of Washington D.C. near well-known landmarks, tourist attractions and residential areas.

In [ ]:
# which day has the most trips
tripsPerDay = df_trips.groupby(df_trips['Start date'].dt.date).size()
print("Day with most trips: %s (%d trips)" % (tripsPerDay.idxmax(), tripsPerDay.max()))

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go

# initialize plotly for offline use in Jupyter Notebooks
init_notebook_mode(connected=True)

# create Year_month column for grouping
df_trips['Year_month'] = df_trips['Start date'].dt.strftime('%Y_%m')

# plot number of trips per month
graphData = df_trips.groupby('Year_month').size()

data = [go.Bar(
    x=graphData.index,
    y=graphData.values
)]

iplot(data)

***Conclusion***\
This was a first impression of the bikesharing dataset. A clear upward trend in usage over time can be seen and the service is used most intensively in the central areas of Washington D.C.

## Trip Duration by Member Type

In [ ]:
data = []

# plot histogram of trip durations for each member type (only trips shorter than 60 minutes)
for memberType in df_trips['Member type'].drop_duplicates():
    data.append(go.Histogram(x = round(df_trips[(df_trips['Member type'] == memberType) & (df_trips['Duration'] <= 60 * 60)]['Duration'] / 60), name = memberType))

iplot(data)

<img src="img/price.png"/>
<img src="img/member.png"/>

Members use the service more intensively than Casual users. This is mainly due to the lower costs for Members. It also becomes clear that Members mostly rely on the service for short rides, while Casual users tend to take medium or longer trips.

## Trips per Hour with Interactive Filters

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# options for day filter
DAY_OPTS = ['All', 'Working days', 'Weekend / Holidays']
weekend = {'Sat', 'Sun'}

# widgets
member_opts = ['All'] + sorted(df_trips['Member type'].dropna().unique().tolist())
w_member = widgets.Dropdown(options=member_opts, value='All', description='Filter:')
w_day    = widgets.RadioButtons(options=DAY_OPTS, value='All', description='Days:')

# plot
fig = go.FigureWidget([go.Bar(x=list(range(24)), y=[0]*24)])
fig.update_layout(title='Trips per hour', xaxis_title='Hour of day', yaxis_title='Trips')

# function to compute series based on filters
def compute_series(member, dayopt):
    df = df_trips

    # filter by member type
    if member != 'All':
        df = df[df['Member type'] == member]
    if dayopt == 'Working days':
        df = df[~df['Weekday'].isin(weekend) & (df['Holiday'] != 1)]
    elif dayopt == 'Weekend / Holidays':
        df = df[df['Weekday'].isin(weekend) | (df['Holiday'] == 1)]

    s = df.groupby(df['Start date'].dt.hour).size()
    return s.reindex(range(24), fill_value=0).sort_index()

# update function for widgets
def update(*_):
    s = compute_series(w_member.value, w_day.value)
    fig.data[0].y = s.values

# attach update function to widgets
w_member.observe(update, names='value')
w_day.observe(update, names='value')

# display widgets and plot
display(w_day, w_member, fig)
update()

The analysis of these plots shows a clear distinction between Members and Casual users.

Members show distinct peaks during the morning and evening rush hours. The usage climb to almost 900,000 trips in a single hour. This points to the service being heavily used for getting to and from work. The overall shorter trip durations fit well with the suggesting that bikes are primarily chosen for efficiency rather than leisure.

Casual users on the other hand follow a different pattern. On weekdays there are no visible spikes around commuting times and their trips are spread more evenly across the day. This indicates that Casual users rarely depend on the system for work-related travel.

On weekends and holidays both groups behave similarly. The trip distributions overlap closely reflecting that the service is then used more for leisure and recreation rather than commuting.

Overall Members account for the majority of rides especially during working days, while Casuals become more present during non-working days.

## Weather Data Integration

In [ ]:
import zipfile, json
import pandas as pd

# empty list to hold dataframes
dfs = []

# read all JSON files from the ZIP archive
with zipfile.ZipFile("data/weather.zip", "r") as z:
    for name in z.namelist():
        if name.endswith(".json"):
            with z.open(name) as f:
                data = json.load(f)['daily']['data']
                dfs.append(pd.DataFrame(data))

# concatenate all dataframes into a single dataframe
df_weather = pd.concat(dfs, ignore_index=True, sort=True)
# convert 'time' column from epoch to datetime
df_weather['time'] = pd.to_datetime(df_weather['time'], unit='s')

df_weather

## Data Cleaning and Conversion

Relevant columns are converted from Fahrenheit to Celsius and corrupted records with missing values are removed.

In [ ]:
# convert columns in celsius
convertToCelsius = ['apparentTemperatureHigh', 'apparentTemperatureLow', 'apparentTemperatureMax', 'apparentTemperatureMin', 'temperatureHigh', 'temperatureLow', 'temperatureMax', 'temperatureMin']

# convert relevant columns from Fahrenheit to Celsius
for col in convertToCelsius:
    df_weather[col] = df_weather[col].apply(lambda F: 5/9*(F - 32))

df_weather

In [ ]:
filterForNANs = ['apparentTemperatureHigh', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime', 'apparentTemperatureMax', 'apparentTemperatureMaxTime','apparentTemperatureMin', 'apparentTemperatureMinTime', 'icon', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow', 'temperatureLowTime', 'temperatureMax','temperatureMaxTime', 'temperatureMin', 'temperatureMinTime', 'time']

# determine columns with NANs
df_corrupted = df_weather[df_weather[filterForNANs].isna().any(axis=1)]

# delete NAN rows
df_weather = df_weather[~df_weather[filterForNANs].isna().any(axis=1)]

if len(df_corrupted.index) > 0:
    print("Removed corrupted records:")
    display(df_corrupted)
else:
    print("No corrupted records found.")

## Trips and Weather Conditions

Trips per day are merged with daily weather data. The average number of trips is then grouped by weather condition (icon).

In [ ]:
# calculate number of trips per day
df_tripsPerDay = pd.DataFrame( {'count' : df_trips.groupby(df_trips['Start date'].dt.date).size()}).reset_index().rename(columns={'Start date' : 'date'})
# extract just the date (no time) from weather data
df_weather['date'] = df_weather['time'].dt.date

# merge trips per day with corresponding weather condition
df_tripsPerDay = df_tripsPerDay.merge(df_weather[['icon', 'date']], on='date')
# show average number of trips per weather condition
df_tripsPerDay.groupby('icon', dropna=False)['count'].mean()

In [ ]:
# convert 'date'-column to datetime
df_tripsPerDay['date'] = pd.to_datetime(df_tripsPerDay['date'])

# filter data for the year 2015
graphData = df_tripsPerDay[df_tripsPerDay['date'].dt.year == 2015]

# generate scatterplot with the above data
data = [go.Scatter(x=graphData['date'], y=graphData['count'])]

iplot(data)

**Outliers Explained**

A scatter plot of trips per day in 2015 shows several spikes and drops that can be linked to specific events:
<ul>
    <li><p>
        <b>11.04.15 (High): </b>Cherry Blossom Parade <a href="https://abcnews.go.com/US/cherry-blossoms-washington-dcs-amazing-trees/story?id=30279909">Link</a> 
    </p></li>
    </p></li>
    <li><p>
        <b>14.04.15 (Low): </b>Rain on the last day of the Cherry Blossom Day <a href="https://cherryblossomwatch.com/cherry-blossom-watch-update-april-14-2015/">Link</a> 
    </p></li>
    <li><p>
        <b>18.04.15 (High): </b>Global Citizen Earth Day <a href="http://www.msnbc.com/msnbc/global-citizen-2015-earth-day#slide1">Link</a> 
    </p></li>
    <li><p>
        <b>27.06.15 (Low): </b>Due to a stormy thunderstorm there were fewer cyclists <a href="https://www.washingtonpost.com/news/capital-weather-gang/wp/2015/06/27/severe-storms-possible-today-with-heavy-rain-flooding-strong-winds-updates/?utm_term=.52b28636a272">Link</a>
    </p></li>
    <li><p>
        <b>02.10.15 (Low): </b>Fewer cyclists due to thunderstorm and tornado warnings <a href="https://www.nbcwashington.com/weather/stories/Tornado-Touched-Down-in-Laurel-This-Week-NWS-Confirms-330441911.html">Link</a></p>
    </p></li>
</ul>

## Temperature vs. Number of Trips

A scatter plot comparing maximum daily temperature and number of trips (2015–2017) shows that people ride more often on warm days.
In summer rides are more consistent, while in winter trip counts are more scattered.

In [ ]:
# combine max temperature for 2015-2017
df_tripsPerDay['date'] = df_tripsPerDay['date'].dt.date
tripsPerTemperature = df_weather[['date', 'temperatureMax']].merge(df_tripsPerDay, on='date')

# x axis shows highest temperature and y axis the number of trips in the period 2015-2017
data = [go.Scatter(x=tripsPerTemperature['temperatureMax'], y=tripsPerTemperature['count'], mode="markers")]

iplot(data)

## Correlation
The empirical correlation coefficient between maximum temperature and number of trips is calculated.

In [ ]:
import math

# average values for highest temperature and number of trips
meanTMax     = tripsPerTemperature['temperatureMax'].mean()
meanCount    = tripsPerTemperature['count'].mean()

# initialization of variables and partial calculation for the calculation of the empirical correlation coefficient
zaehlSum     = 0
nennSumLeft  = 0
nennSumRight = 0
for row in tripsPerTemperature.iterrows():
    zaehlSum     += (row[1]['temperatureMax'] - meanTMax)  * (row[1]['count'] - meanCount)
    nennSumLeft  += (row[1]['temperatureMax'] - meanTMax)  ** 2
    nennSumRight += (row[1]['count']          - meanCount) ** 2

# calculation of the empirical correlation coefficient and output
empKorKoef = zaehlSum / math.sqrt(nennSumLeft * nennSumRight)
empKorKoef

Result: **0.807**

This strong positive correlation shows that warmer days lead to significantly more bike trips. The closer the temperature gets to summer levels, the more consistently the service is used, while in colder months usage drops and becomes more irregular.

## Station Imbalance (Arrivals vs. Departures)

In [ ]:
# determine the number of arrivals and departures
df_departure = pd.DataFrame(df_trips.groupby('Start station').size()).reset_index().rename(columns={'Start station' : 'Station', 0 : 'Departure'})
df_arrival = pd.DataFrame(df_trips.groupby('End station').size()).reset_index().rename(columns={'End station' : 'Station', 0 : 'Arrival'})

# combine arrivals and departures with stations
df_arrdeparture = df_arrival.merge(df_departure, on='Station', how='outer').fillna(0)

# calculate the difference to see how many bicycles were taken and returned.
df_arrdeparture['Difference'] = df_arrdeparture['Arrival'] - df_arrdeparture['Departure']

# Sort the table by difference
df_arrdeparture.sort_values(by=['Difference'])

To get an initial understanding of the imbalance a dataframe was created showing for each station whether departures or arrivals dominated over the whole period.

- Negative values → more departures than arrivals
- Positive values → more arrivals than departures

Deficits are mostly found in residential areas, while surpluses in arrivals occur mainly at stations close to office districts and tourist attractions.

## Daily Imbalances per Station

In [ ]:
from ipywidgets import interact

# new dataframe with TripsPerDayPerStation Start
df_tpDpSStart = pd.DataFrame(df_trips.groupby([df_trips.rename(columns={'Start station' : 'Station'})['Station'], df_trips.rename(columns={ 'Start date' : 'Date'})['Date'].dt.date]).size())
df_tpDpSStart = df_tpDpSStart.rename(columns={0 : 'Starts'}).reset_index()

# new dataframe with TripsPerDayPerStation End
df_tpDpSEnd = pd.DataFrame(df_trips.groupby([df_trips.rename(columns={'End station' : 'Station'})['Station'], df_trips.rename(columns={ 'End date' : 'Date'})['Date'].dt.date]).size())
df_tpDpSEnd = df_tpDpSEnd.rename(columns={0 : 'Ends'}).reset_index()

# merge both dataframes
df_tpDpSE = df_tpDpSStart.merge(df_tpDpSEnd, on = ['Station', 'Date']).fillna(0)
df_tpDpSE['Diff'] = df_tpDpSE['Ends'] - df_tpDpSE['Starts']
df_tpDpSE = df_tpDpSE.sort_values(by='Diff')

# dropdown to select station
graphData = {}
dropDownItems =  df_tpDpSE['Station'].drop_duplicates().sort_values()

for station in dropDownItems:
    data = df_tpDpSE[df_tpDpSE['Station'] == station].sort_values(by='Date')
    graphData.update( {station : go.Scatter(x=data['Date'], y=data['Diff'], name=station)})
    
def dropDownChanged(selection):
    iplot([graphData[selection]])

dropDown = interact(dropDownChanged, selection = dropDownItems)

Breaking down the data further a scatterplot was created showing the daily imbalance per station. A dropdown menu allows filtering by station making it possible to observe the effect of imbalances across the whole period. This also highlights the impact of nearby events and allows outliers to be explained.

- Jan 21, 2017 (High): Women’s March on Washington causing a spike at E St SW & 4th St SW close to Independence Ave SW. Other nearby stations were also affected.

- Apr 11, 2015 (High): Strong spike visible at Ohio Dr & West Basin Dr SW.

- Sep 16, 2017 (High): Gathering of Trump supporters at the Washington Mall.

## Conclusion and Recommendations

The analysis shows that the Capital Bikeshare system experiences clear imbalances across its stations.

- Residential areas often show deficits with more departures than arrivals.

- Office districts and tourist locations typically show surpluses as many riders end their trips there.

- Events such as marches, parades, or political gatherings can cause strong short-term spikes at nearby stations.

**Recommendations:**

- Dynamic bike redistribution: Move bikes proactively from residential areas to central office/tourist stations in the morning and shift them back in the evening.

- Event-based planning: Identify major events in advance (e.g. parades, political rallies, festivals) and pre-distribute bikes to nearby stations.

- Pricing incentives: Offer reduced evening rates or special promotions for members to encourage bike use for return trips helping to reduce deficits at residential stations.

- Introduce e-bikes: Providing more e-bikes can motivate riders to take bikes for the way home instead of switching to public transport when they are tired.

Overall these measures can reduce imbalances and improve availability for all users. Also the service gets more reliable.